In [4]:
import os
import pandas as pd
import numpy as np
from os.path import join as PJOIN
from collections import Counter
import xlsxwriter as xl

In [12]:
#for each annotatation_file

ANNOTATIONS_FILE = "DATA/GENERATED/annotation_gtest.csv"
leftovername = ANNOTATIONS_FILE[len("DATA/GENERATED_NEW/"):].replace("/","_")
outputfilename = "ANALYSIS/DIFF/"+leftovername

# PROJECT_NAME = "OpenCPN"
# AUTHOR_NAME = "allmarked"
# OUTPUT_DIR = "DATA/GENERATED/TRAIN/"
# OUTPUT_SUFFIX = "_cal"
# OUTPUT_FILENAME = "train_" + PROJECT_NAME +"_" + AUTHOR_NAME + OUTPUT_SUFFIX + ".csv"
ANNOTATIONS = {'N':1,'P':2,'U':3,'n':1}
REVERSE_MAP = {1: 'N', 2: 'P', 3:'U'}
INTUITIVE_INDEX = 46
CALCULATED_INDEX = 46

In [13]:
if ANNOTATIONS_FILE[-3:] == 'csv':
    annotations_file = pd.read_csv(ANNOTATIONS_FILE,delimiter='$')
else:
    annotations_file = pd.read_excel(ANNOTATIONS_FILE)
annotations_file.head()

,Filename,Comment text,Start line,End line,No. of words,Program Domain Concepts,Problem Domain Concepts,Copyright/License,Bug/Fix/Patch/Version,Build,...,C24,C25,C26,C27,C28,C29,C30,Calculated Score New,Matching Rules,Comment Type
0,googletest/googletest/src/gtest-all.cc/gtest-a...,"Copyright 2008, Google Inc.\nAll rights reserv...",1,28,222,"{'rights': ['right', 'Data-Structure and its C...","['event', 'google']",True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,"1,16","[1, 1, 0]"
1,googletest/googletest/src/gtest-all.cc/gtest-a...,\nGoogle C++ Testing and Mocking Framework (Go...,30,34,25,"{'testing': ['reseting', 'Operations as part o...","['file', 'mock', 'google', 'test']",False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,"1,19","[1, 1, 1]"
2,googletest/googletest/src/gtest-all.cc/gtest-a...,This line ensures that gtest.h can be compiled...,36,37,15,"{'ensures': ['enquires', 'Operations as part o...","['gtest', 'test']",False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,1,"[1, 1, 0]"
3,googletest/googletest/src/gtest-all.cc/gtest-a...,The following lines pull in the real gtest *.c...,40,40,10,"{'real': ['real', 'Data-Structure and its Comp...","['files', 'file', 'gtest', 'test']",False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,1,"[1, 1, 0]"
4,googletest/googlemock/src/gmock-all.cc/gmock-a...,"Copyright 2008, Google Inc.\nAll rights reserv...",1,28,222,"{'rights': ['right', 'Data-Structure and its C...","['event', 'google']",True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,"1,16","[1, 1, 0]"


In [14]:
annotations_np = np.array(annotations_file)
for i,el in enumerate(annotations_np[0]):
    print(i," : ",el)

0  :  googletest/googletest/src/gtest-all.cc/gtest-all.cc
1  :  Copyright 2008, Google Inc.
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are
met:

* Redistributions of source code must retain the above copyright
notice, this list of conditions and the following disclaimer.
* Redistributions in binary form must reproduce the above
copyright notice, this list of conditions and the following disclaimer
in the documentation and/or other materials provided with the
distribution.
* Neither the name of Google Inc. nor the names of its
contributors may be used to endorse or promote products derived from
this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
"AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR
A PARTICULAR PURPOSE ARE

In [15]:
num_nan = 0
labels_int = []
labels_cal = []
all_data = []
for comments_data in annotations_np:
    labels_intuitive = comments_data[INTUITIVE_INDEX]
    labels_calculated = comments_data[CALCULATED_INDEX]
    if labels_intuitive != labels_intuitive:
        num_nan += 1
        continue
    if labels_calculated != labels_calculated:
        print("Calculated Label not present")
        print(comments_data)
    labels_int.append(ANNOTATIONS[labels_intuitive[0]])
    labels_cal.append(ANNOTATIONS[labels_calculated[0]])    
    all_data.append(comments_data)
labels_int = np.array(labels_int)
labels_cal = np.array(labels_cal)

In [16]:
print(os.path.basename(ANNOTATIONS_FILE)),print(num_nan), print(len(labels_int)), print(np.sum(labels_int==labels_cal)), print(np.sum(labels_int==labels_cal)*100/len(labels_int))

annotation_gtest.csv
0
14
14
100.0


(None, None, None, None, None)

In [17]:
confusion = Counter()
examples = {}
for i in range(len(labels_int)):
    key = (REVERSE_MAP[labels_int[i]],REVERSE_MAP[labels_cal[i]])
    confusion[key] += 1
    if key not in examples:
        examples[key] = []
    examples[key].append(all_data[i])

In [18]:
confusion

Counter({('U', 'U'): 12, ('P', 'P'): 1, ('N', 'N'): 1})

In [19]:
examples

{('U',
  'U'): [array(['googletest/googletest/src/gtest-all.cc/gtest-all.cc',
         'Copyright 2008, Google Inc.\nAll rights reserved.\n\nRedistribution and use in source and binary forms, with or without\nmodification, are permitted provided that the following conditions are\nmet:\n\n* Redistributions of source code must retain the above copyright\nnotice, this list of conditions and the following disclaimer.\n* Redistributions in binary form must reproduce the above\ncopyright notice, this list of conditions and the following disclaimer\nin the documentation and/or other materials provided with the\ndistribution.\n* Neither the name of Google Inc. nor the names of its\ncontributors may be used to endorse or promote products derived from\nthis software without specific prior written permission.\n\nTHIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS\n"AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT\nLIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILI

In [20]:
wb = xl.Workbook(outputfilename)
columns = annotations_file.columns
for k,v in examples.items():
    sheet_name = k[0] + "," + k[1]
    ws = wb.add_worksheet(sheet_name)
    for i in range(len(columns)):
        if columns[i] != columns[i]:
            continue
        ws.write(0,i,columns[i])
    for i,el in enumerate(v):
        for j,e in enumerate(el):
            if e != e:
                continue
            ws.write(i+1,j,e)
wb.close()